In [19]:
import chess
import chess.pgn
from io import StringIO
import stockfish
import re
import pandas as pd

In [122]:
games = 0
game = []
after_moves = False
after_pgn = False
parsed_games = []
firstline = True
game_infos = []
sf = stockfish.Stockfish('./stockfish-10-win/Windows/stockfish_10_x64.exe',
                         depth=8)
df = pd.DataFrame([],
                  dtype='int16',
                  columns=['game_link',
                           'white',
                           'black',
                           'sf_rating',
                           'end',
                           'time_control',
                           'white_elo',
                           'black_elo'])
start = time.time()
with open('F:/lichess/august_games.pgn', 'r') as f:
    for line in f:
        game.append(line)
        if line == '\n':
            if after_moves:
                after_pgn = True
                after_moves = False
            else:
                after_moves = True
        if after_pgn:
            parsed_game = chess.pgn.read_game(StringIO(''.join(game)))
            game = []
            games += 1
            after_pgn = False
            
            fen = parsed_game.end().board().fen()
            end = parsed_game.headers['Termination']
            game_link = parsed_game.headers['Site'][20:]
            black = parsed_game.headers['Black']
            white = parsed_game.headers['White']
            time_control = parsed_game.headers['TimeControl']
            white_elo = int(parsed_game.headers['WhiteElo'])
            black_elo = int(parsed_game.headers['BlackElo'])
            
            sf.set_fen_position(fen)
            sf.get_best_move()
            info_string = sf.info
            
            rating_match = re.search(r'score (cp|mate) (.+?)(?: |$)',info_string)
            if rating_match.group(1) == 'mate':
                original_rating = int(rating_match.group(2))
                if original_rating:
                    rating = 9999 * original_rating / abs(original_rating)
                elif parsed_game.headers['Result'] == '1-0':
                    rating = 9999
                else:
                    rating = -9999
            else:
                rating = int(rating_match.group(2))
            if ' b ' in fen:
                rating *= -1
            game_infos.append([game_link, white, black, rating, end, time_control, white_elo, black_elo])
        if games % 10:
            new_df = pd.DataFrame(game_infos,
                                  dtype='int16',
                                  columns=['game_link',
                                           'white',
                                           'black',
                                           'sf_rating',
                                           'end',
                                           'time_control',
                                           'white_elo',
                                           'black_elo'])
            df = pd.concat([df, new_df], axis=0, sort=False)
            game_infos = []
        if games > 100:
            break
print(time.time() - start)

10.899963855743408


In [123]:
df.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
Int64Index: 101 entries, 0 to 1
Data columns (total 8 columns):
game_link       101 non-null object
white           101 non-null object
black           101 non-null object
sf_rating       101 non-null int16
end             101 non-null object
time_control    101 non-null object
white_elo       101 non-null int16
black_elo       101 non-null int16
dtypes: int16(3), object(5)
memory usage: 33.4 KB


In [124]:
df.head()

,game_link,white,black,sf_rating,end,time_control,white_elo,black_elo
0,EeKb62cG,Karnak13,Sergiomeira8,-384,Time forfeit,120+1,1752,1534
0,y1vpWdnP,Princepe28,HojeNao,-954,Normal,120+1,1832,1861
0,FDuEtjq7,egypt7050,skippyboy,-9999,Normal,360+0,1219,1856
0,fMJIkFIV,elsadchess,Paparama,195,Time forfeit,180+0,2365,2480
0,1PsHgvDI,manesht,swinegobbler,-30,Time forfeit,300+0,1804,1766


In [125]:
approx_mem_usage = ((36.75 * 1000000)  # game count
                    * 33.4  # memory usage for 100 rows
                    / 100  # 100 rows
                   )
approx_mem_usage / (1024 * 1024)  # in gb

11.705875396728516

In [48]:
sf = stockfish.Stockfish('./stockfish-10-win/Windows/stockfish_10_x64.exe',
                         depth=22)

In [7]:
sf.set_fen_position('r3k3/pp3pb1/8/2p1P3/2Pp1pN1/1P1P4/3NRP2/2q1K3 w q - 1 26')

In [52]:
sf.set_position(['e2e4'])

In [8]:
sf.get_best_move()
sf.info

'info depth 18 seldepth 15 multipv 1 score cp 0 nodes 2058685 nps 2462541 tbhits 0 time 836 pv h5e5 f7f6 b2b4 d6e6 e5d4 g7f7 d4a7 f7f8 b4b5 f6g6 g2f3 g6f6 f3g3 f6g6'

In [60]:
info_str = 'info depth 22 seldepth 36 multipv 1 score cp 1493 nodes 552164 nps 3119570 tbhits 0 time 177 pv b5b4 b7f7 g7f7 g3f4 g5g4 h3g4 b4b3 f4e5 b3b2 e5d5 b2b1q d5e5 b1d3 d4d5 d3e3 d5d6 e3d3 e5f4 e4e3 f4f3 e3e2 f3f2 d3e4 f2e1 e4g4'

In [64]:
match = re.search(r'score (cp|mate) (.+?) ',info_str)

In [65]:
match.group(1)

'1493'